In [ ]:
import os

import pandas as pd
import numpy as np
import shap

from rdkit.Chem.rdmolfiles import MolFromSmiles
from rdkit.Chem.Draw import SimilarityMaps
from rdkit.Chem.AllChem import GetMorganGenerator, AdditionalOutput

from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

from modelTools import train_and_test_model, shap_frequency_raw

## Preapre the dataset

In [ ]:
# predefine the dataset file name
dataset_filename = {
    "RO" : "Data/P25_RO_ComVersion_OnlyPhotocat.xlsx",
    "EtOH" : "Data/P25_ETOH_ComVersion_Comprehensive.xlsx",
    "CHCl3" : "Data/P25_CHCl3_ComVersion_Comprehensive.xlsx"
}

### Import the data

In [ ]:
dataset_name = "RO" # dataset name only can be "RO", "EtOH", "CHCl3"

In [ ]:
filename = dataset_filename[dataset_name]
current_dir = os.getcwd()
raw_data_file = os.path.join(current_dir, filename)
raw_data_excel = pd.read_excel(raw_data_file)

raw_data_excel

### Data Curation

In [ ]:
#Drop Columns not exposed to models
column_drop = ["Catalyst", "Count", "n1", "n2"]

#Remove Reso data, removal % could not be accurately measured
cleaned_dataset = raw_data_excel[raw_data_excel['Dye'] != "Reso"]

#Remove Photolysis and Pre-Irradiation Time 0 measurements
cleaned_dataset = cleaned_dataset[cleaned_dataset['Catalyst'] != 0]

cleaned_dataset = cleaned_dataset.drop(columns=column_drop)

cleaned_dataset_drop_T0 = cleaned_dataset[cleaned_dataset['Rxn Time (min)'] != 0]

# If removal value is negative, set it to 0
cleaned_dataset_drop_T0['Removal %'] = cleaned_dataset_drop_T0['Removal %'].apply(lambda x: 0 if x < 0 else x)

cleaned_dataset_drop_T0

In [ ]:
#Define a Helper function to generate morgan fingerprints from dye structures
def get_morgan_fp(dye_smiles_dict: dict, radius: int = 2, n_bits: int = 1024):
    """
    Get Morgan Fingerprints for the dyes with the given SMILES strings, radius, n_bits, use_chirality, use_features

    Return the molecule fingerprint dict, molecule fingerprint list dict and bit dict
    """
    # overall dict
    dye_morgan_fp_dict = {}
    dye_morgan_fp_bit_dict = {}
    dye_morgan_fp_lst_dict = {}

    fp_gen = GetMorganGenerator(radius=radius, fpSize=n_bits)
    ao = AdditionalOutput()
    ao.CollectBitInfoMap()

    for dye, smiles in dye_smiles_dict.items():
        mol = MolFromSmiles(smiles)
        fp = fp_gen.GetFingerprint(mol, additionalOutput=ao)

        dye_morgan_fp_dict[dye] = fp
        dye_morgan_fp_lst_dict[dye] = np.array(fp)
        dye_morgan_fp_bit_dict[dye] = ao.GetBitInfoMap()

        print(f"The Dye is: {str(dye)}, non-zero elements: {np.count_nonzero(np.array(fp))}")    
    
    return dye_morgan_fp_dict, dye_morgan_fp_lst_dict, dye_morgan_fp_bit_dict

In [ ]:
#Initialize a dictionary to hold the dye strings; Dye Abbreviations are keys, value is dye's SMILE string
dye_smiles_dict = {}
for index, row in cleaned_dataset_drop_T0.iterrows():
    dye_smiles_dict[row['Dye']] = row['SMILES']

dye_smiles_dict

In [ ]:
#Prepare the list of possible Morgan fingerprint 
radius_lst = [2]
n_bits_lst = [4096]

assert len(radius_lst) == len(n_bits_lst)

In [ ]:
# generate the morgan fingerprints for the dyes with different parameters and store them in the dict, 
# key is the parameter tuple, value is the morgan fingerprints dict:  molecule fingerprint dict, molecule fingerprint list dict and bit dict

#Originally intedended to compare different values of radius and number of bits, only implemented for single radius and n_bits (2,4096)

morgan_fp_dict = {}
for i in range(len(radius_lst)):
    morgan_fp_dict[(radius_lst[i], n_bits_lst[i])] = (get_morgan_fp(dye_smiles_dict, radius_lst[i], n_bits_lst[i]))
    

In [ ]:
#Store Morgan Fingerprints in dataframe for later use
smile_morgan_fp = pd.DataFrame(morgan_fp_dict[(2, 4096)][1]).T

In [ ]:
# add morgan fingerprint into raw dataset based on the dye name
clean_feature_dropT0_dataset_full = cleaned_dataset_drop_T0.join(smile_morgan_fp, on='Dye')

# set name to string for later training purpose
clean_feature_dropT0_dataset_full.columns = clean_feature_dropT0_dataset_full.columns.astype(str)

### Test/Train Split


In [ ]:
#For later purposes, copy dataset with dye column remaining at the beginning
clean_feature_dropT0_dataset = clean_feature_dropT0_dataset_full.drop(columns=["SMILES", "Std Dev"])
target_name = "Removal %"


In [ ]:
clean_feature_dropT0_dataset

In [ ]:
#For leave out sets, select which dye should be removed from training data, if all dyes are to be included, pass empty string -> ""
remove_dye_name = "Am"

removed_dye_dataset = clean_feature_dropT0_dataset[clean_feature_dropT0_dataset['Dye'] == remove_dye_name]
clean_feature_dropT0_dataset = clean_feature_dropT0_dataset[clean_feature_dropT0_dataset['Dye'] != remove_dye_name]

clean_feature_dropT0_dataset

In [ ]:
#Prepare datasets of the properties and targets
properties_set = clean_feature_dropT0_dataset.drop(columns=target_name)
properties_name = properties_set.columns.values
target_set = clean_feature_dropT0_dataset[target_name]

In [ ]:
#For later analysis purposes, do not drop dye column before split
X_train, X_test, Y_train, Y_test = train_test_split(properties_set, target_set, test_size=0.2, random_state=16)

In [ ]:
#If a dye was removed, re-add its data into the test dataset
if remove_dye_name != "":
    X_test = pd.concat([X_test, removed_dye_dataset.drop(columns=target_name)])
    Y_test = pd.concat([Y_test, removed_dye_dataset[target_name]])
    print(f"Add removed dye {remove_dye_name} data into the test dataset")
else:
    print("No dye is removed, no need to add removed dye data into the test dataset")


In [ ]:
# drop the dye column for all and store the dye column for later check dye purpose
X_train_dye = pd.DataFrame(X_train.pop("Dye"), columns=["Dye"])
X_test_dye = pd.DataFrame(X_test.pop("Dye"), columns=["Dye"])

## Model Preparations


### Linear Regression

In [ ]:
model_use_LR = LinearRegression
model_params_LR = {}

### Random Forest

In [ ]:
model_use_RF = RandomForestRegressor
model_params_RF = {"max_depth": 10, "n_estimators":500, "oob_score":True,  "random_state":17}

### Neural Network

In [ ]:
model_use_NN = MLPRegressor
model_params_NN = {'solver': 'lbfgs', "max_iter": 20000, 'learning_rate': 'invscaling', 'hidden_layer_sizes': (20, 50), 'batch_size': 20, 'alpha': 0.01, 'activation': 'logistic'}

In [ ]:
#Create a dict to store the model for later use; key is the model name, value is a tuple with model and model parameters
model_dict = {
    "LR" : (model_use_LR, model_params_LR),
    "RF" : (model_use_RF, model_params_RF),
    "NN" : (model_use_NN, model_params_NN)
}

## Train the model

### Train a single model

In [ ]:
#Define which model should be used for training
model_use_name = "LR" # can only be "LR", "RF", "NN"

In [ ]:
#Define the model that will be trained and tested
model_use = model_dict[model_use_name][0]
model_params = model_dict[model_use_name][1]    

model_use_single = model_use(**model_params)

model_use_single

In [ ]:
#Use the model tools helper file to train and test the model, as well as perform cross validation
model_use_single, msg = train_and_test_model(model_use_single, f"{model_use_name} model", X_train, Y_train, X_test, Y_test, cv=10)
print(msg)

In [ ]:
#Test the model with the removed dye
if remove_dye_name != "":
    
    removed_dye_properties = removed_dye_dataset.drop(columns=[target_name, "Dye"])
    removed_dye_target = removed_dye_dataset[target_name]

    removed_dye_predicted_values = model_use_single.predict(removed_dye_properties)

    print(f"The predicted values for the removed dye are {list(removed_dye_predicted_values)}")
    print(f"The actual values for the removed dye are {list(removed_dye_target)}")
else:
    print("No dye was removed, so no need to test the removed dye")

In [ ]:
#Convivient pause to check model performance
raise RuntimeError("Stop here")

# SHAP Value Assignments

## SHAP value for single model

In [ ]:
#Define helpers to explain the SHAP values of a trained model
explainer = shap.Explainer(model_use_single.predict, X_train)
shap_values = explainer(X_train, max_evals=1000)

In [ ]:
#Output top 20 most important SHAP values
shap.plots.beeswarm(shap_values, max_display=20)

In [ ]:
shap.plots.bar(shap_values, max_display=20)

In [ ]:
shap.plots.bar(shap_values.abs.max(0), max_display=20)

## SHAP value for multiple tests (Raw data)

In [ ]:
#Re-run models with different random seeds to variability of the model
# set parameters
random_seed = 16
split_random_num = 5
model_random_num = 10

#Since linear regression is not random, so model_random_num is None and give model_random_list = [None] instead and change split_random_num to the multiple of split_random_num and model_random_num
if model_use_name == "LR":
    model_random_list = [None]
    split_random_num = split_random_num * model_random_num
    model_random_num = None
else:
    model_random_list = None

# Use make/train test set 
properties_use = properties_set
target_use = target_set

In [ ]:
#Create a dataframe of SHAP values produced by models with varied random seeds
shap_value_raw_df, msg = shap_frequency_raw(model_use, model_params, properties_use, target_use, data_split_random_num=split_random_num, model_random_num=model_random_num, model_random_list=model_random_list, random_seed=random_seed)

In [ ]:
print(msg)

In [ ]:
# drop the columns that all values in this column are 0 to reduce the size of the dataframe
shap_value_raw_df = shap_value_raw_df.loc[:, (shap_value_raw_df != 0).any(axis=0)]

In [ ]:
# save the data
shap_value_raw_df.to_csv(f"shap_value_raw_all_drop_{model_use_name}_{dataset_name}.csv", index=False)

In [ ]:
raise RuntimeError("Stop here")